In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

garage_data = pd.read_csv('/content/drive/MyDrive/decodeX/garage_data.csv')
surveyor_data = pd.read_csv('/content/drive/MyDrive/decodeX/surveyor_data.csv')

In [ ]:
print("Garage Data Sample:")
garage_data.head()

Garage Data Sample:


,REFERENCE_NUM,PRODUCT_INDEX,VEHICLE_MODEL_CODE,CLAIMNO,CLAIM_NO,PARTNO,PARTDESCRIPTION,TOTAL_AMOUNT,NUM_CLAIM_NO
0,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,64520M56R00,"PANEL COMP, RR LAMP HOUSE, L",285.15,2.300203e+19
1,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,84681M56R00,"WEATHERSTRIP, TRUNK LID OPNG",484.37,2.300203e+19
2,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,65700M56R00,PANEL ASSY TRUNK LID,2492.18,2.300203e+19
3,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,35750M56R00,"LAMP, UNIT RR COMB LH",1915.25,2.300203e+19
4,202012250082721,N000706,30135.0,'23002031863800000001,C230020318638,31100M86J30,"MOTOR ASSY, STARTING",3613.28,2.300203e+19


In [ ]:
garage_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365086 entries, 0 to 365085
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   REFERENCE_NUM       365086 non-null  int64  
 1   PRODUCT_INDEX       365086 non-null  object 
 2   VEHICLE_MODEL_CODE  365086 non-null  float64
 3   CLAIMNO             365086 non-null  object 
 4   CLAIM_NO            365086 non-null  object 
 5   PARTNO              365086 non-null  object 
 6   PARTDESCRIPTION     365086 non-null  object 
 7   TOTAL_AMOUNT        365086 non-null  float64
 8   NUM_CLAIM_NO        365086 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 25.1+ MB


In [ ]:
garage_data.shape

(365086, 9)

In [ ]:
garage_data['REFERENCE_NUM'].head()



,REFERENCE_NUM
0,202001050026449
1,202001050026449
2,202001050026449
3,202001050026449
4,202012250082721


In [ ]:
surveyor_data.shape

(945216, 9)

In [ ]:
surveyor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945216 entries, 0 to 945215
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   REFERENCE_NUM         945216 non-null  int64  
 1   PRODUCT_INDEX         945216 non-null  object 
 2   VEHICLE_MODEL_CODE    945016 non-null  float64
 3   CLAIMNO               945216 non-null  object 
 4   NUM_CLAIM_NO          945216 non-null  float64
 5   TXT_PARTS_GROUP_NAME  945216 non-null  object 
 6   TXT_PARTS_NAME        945216 non-null  object 
 7   TOTAL_AMOUNT          945216 non-null  float64
 8   NUM_PART_CODE         945216 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 64.9+ MB


In [ ]:
print("\nSurveyor Data Sample:")
surveyor_data.head()


Surveyor Data Sample:


,REFERENCE_NUM,PRODUCT_INDEX,VEHICLE_MODEL_CODE,CLAIMNO,NUM_CLAIM_NO,TXT_PARTS_GROUP_NAME,TXT_PARTS_NAME,TOTAL_AMOUNT,NUM_PART_CODE
0,202108260089768,N000706,23139.0,'23002200306800000001,2.300220e+19,Rubber Plastic Parts,Emblem|Logo Rear Middle,189.05,1086
1,202107090087260,N000706,38123.0,'23002149113800000001,2.300215e+19,Rubber Plastic Parts,Bumper Rear Assembly,1863.00,1007
2,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Glass Parts,Windshield Glass Front,2860.00,1003
3,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Rubber Plastic Parts,Moulding Front Windshield,296.00,1035
4,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Metalic Parts,Bonnet|Hood Assembly,2628.00,1002


In [ ]:
import numpy as np

# Standardize CLAIMNO column in both datasets
garage_data['CLAIMNO'] = garage_data[['CLAIMNO', 'CLAIM_NO', 'NUM_CLAIM_NO']].bfill(axis=1).iloc[:, 0]
surveyor_data['CLAIMNO'] = surveyor_data[['CLAIMNO', 'NUM_CLAIM_NO']].bfill(axis=1).iloc[:, 0]

# Drop rows with no CLAIMNO or REFERENCE_NUM
garage_data.dropna(subset=['CLAIMNO', 'REFERENCE_NUM'], inplace=True)
surveyor_data.dropna(subset=['CLAIMNO', 'REFERENCE_NUM'], inplace=True)

# Merge on both CLAIMNO and REFERENCE_NUM
merged_data = pd.merge(garage_data, surveyor_data,
                       on=['CLAIMNO', 'REFERENCE_NUM'],
                       suffixes=('_garage', '_surveyor'))

# Select and rename relevant columns
selected_columns = ['CLAIMNO',
                    'REFERENCE_NUM',
                    'PRODUCT_INDEX_garage',
                    'PARTDESCRIPTION',
                    'TXT_PARTS_GROUP_NAME',
                    'TXT_PARTS_NAME']
merged_data = merged_data[selected_columns].copy()
merged_data.rename(columns={'PRODUCT_INDEX_garage': 'PRODUCT_INDEX'}, inplace=True)

# Preprocess text columns: convert to lowercase and remove special characters
# Preprocess text columns: convert to lowercase and remove special characters and numbers
text_columns = ['PARTDESCRIPTION', 'TXT_PARTS_GROUP_NAME', 'TXT_PARTS_NAME']
merged_data[text_columns] = merged_data[text_columns].apply(lambda x: x.str.lower()
                                                                         .str.replace('[0-9]', '', regex=True)  # Remove numbers
                                                                         .str.replace('[^a-z ]', '', regex=True))  # Remove special characters

# Update selected_columns to reflect the renamed column
selected_columns = ['CLAIMNO',
                    'REFERENCE_NUM',
                    'PRODUCT_INDEX',
                    'PARTDESCRIPTION',
                    'TXT_PARTS_GROUP_NAME',
                    'TXT_PARTS_NAME']

# Remove rows with any null values in the selected columns
merged_data.dropna(subset=selected_columns, inplace=True)

# Display a sample of the preprocessed merged data
merged_data.head()




,CLAIMNO,REFERENCE_NUM,PRODUCT_INDEX,PARTDESCRIPTION,TXT_PARTS_GROUP_NAME,TXT_PARTS_NAME
0,'23002017715900000001,201911010097702,N000706,joint compfr stabilizer bar,glass parts,windshield glass front
1,'23002017715900000001,201911010097702,N000706,module assyairbagchrome,glass parts,windshield glass front
2,'23002017715900000001,201911010097702,N000706,panelfront fenderl,glass parts,windshield glass front
3,'23002017715900000001,201911010097702,N000706,coverengine under sider,glass parts,windshield glass front
4,'23002017715900000001,201911010097702,N000706,module assypassenger air bag,glass parts,windshield glass front


In [ ]:
merged_data.shape

(805370, 6)

In [ ]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.2 MB/s eta 0:00:00


In [ ]:
from rapidfuzz import process

# Clean and normalize TXT_PARTS_NAME
merged_data['TXT_PARTS_NAME'] = merged_data['TXT_PARTS_NAME'].str.lower().str.strip()

# Get unique part names
unique_part_names = merged_data['TXT_PARTS_NAME'].unique()

# Function to find best match for each part name
def find_best_match(part_name, choices):
    best_match = process.extractOne(part_name, choices, score_cutoff=85)
    return best_match[0] if best_match else part_name

# Create a mapping dictionary
mapping_dict = {}
for part in unique_part_names:
    # Map to the best match in unique part names
    mapping_dict[part] = find_best_match(part, unique_part_names)

# Replace with standardized names
merged_data['TXT_PARTS_NAME'] = merged_data['TXT_PARTS_NAME'].map(mapping_dict)

# Group by cleaned TXT_PARTS_NAME and collect all PARTDESCRIPTIONs as a list
grouped_df = merged_data.groupby('TXT_PARTS_NAME')['PARTDESCRIPTION'].apply(list).reset_index()

# Remove duplicates where PARTDESCRIPTION_LIST has all identical values
def all_same(lst):
    return all(x == lst[0] for x in lst)

# Keep only one TXT_PARTS_NAME if all descriptions are same
unique_lists = {}
for _, row in grouped_df.iterrows():
    desc_list = row['PARTDESCRIPTION']
    # Convert to tuple (since list can't be used as dict key)
    desc_tuple = tuple(desc_list)

    # If all values in the list are same and not already in unique_lists
    if all_same(desc_list) and desc_tuple not in unique_lists:
        unique_lists[desc_tuple] = row['TXT_PARTS_NAME']
    elif desc_tuple not in unique_lists:
        unique_lists[desc_tuple] = row['TXT_PARTS_NAME']

# Create a DataFrame from the unique lists
final_df = pd.DataFrame({
    'TXT_PARTS_NAME': list(unique_lists.values()),
    'PARTDESCRIPTION_LIST': [list(tpl) for tpl in unique_lists.keys()]
})

# Display the result
final_df.head()


,TXT_PARTS_NAME,PARTDESCRIPTION_LIST
0,a c can,"[tank assywater reserve, bumperrear, condenser..."
1,a c can gas,[bumperrear upper]
2,a pillar front inner left,"[member fr bumper upper, unit head lamp lh, sc..."
3,a pillar front inner right,"[member fr bumper upper, mounting compeng rr, ..."
4,a pillar front left,"[weatherstripfront door outl, emblem vvt chrom..."


In [ ]:
# Define output path
output_path = '/content/grouped_part_descriptions3.csv'  # Colab ke liye

# Save to CS
grouped_df.to_csv(output_path, index=False)

print(f"✅ CSV file saved successfully at: {output_path}")



✅ CSV file saved successfully at: /content/grouped_part_descriptions3.csv
